In [1]:
import matplotlib.pyplot as plt

In [2]:
import rospy
rospy.init_node('task_planner_2', anonymous=True)

In [3]:
import numpy as np
from pkg.utils import *
import socket
import time as timer
import json

In [4]:
from pkg.panda_repeater import *

In [5]:
panda = PandaRepeater()

In [6]:
panda.send_qval(panda.qcur)

[0.14551763236522675,
 -0.3997618556022644,
 -0.1751592755317688,
 -2.0162971019744873,
 0.06706132739782333,
 2.2000842094421387,
 0.6896503567695618]

In [7]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [12]:
traj_sample = []
DELTA = np.deg2rad(1)
N_steps = 100
DT = 1e-2
t0 = timer.time()
qval0 = panda.get_qcur()
for iii in range(100):
    gtimer.tic("step")
    while timer.time()-t0<DT:
        timer.sleep(1e-3)
    t0 = timer.time()
    
    qval = np.add(qval0,DELTA*iii/N_steps).tolist()
    traj_sample += [panda.send_qval(qval)]
    gtimer.toc("step")
print(gtimer)

step: 	3217.0 ms/300 = 10.723 ms 



In [13]:
traj_sample = []
DELTA = np.deg2rad(-1)
N_steps = 100
DT = 1e-2
t0 = timer.time()
qval0 = panda.get_qcur()
for iii in range(100):
    gtimer.tic("step")
    while timer.time()-t0<DT:
        timer.sleep(1e-3)
    t0 = timer.time()
    
    qval = np.add(qval0,DELTA*iii/N_steps).tolist()
    traj_sample += [panda.send_qval(qval)]
    gtimer.toc("step")
print(gtimer)

step: 	4294.0 ms/400 = 10.734 ms 



In [14]:
panda.move_finger(True)

True

In [15]:
panda.move_finger(False)

False